# Week 6: Data Retrieval and Processing

This notebook retrieves all stored IoT records from the smart contract, processes and cleans the data, and saves the output as a CSV for further analysis.


In [14]:
from web3 import Web3

# Connect to local blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))


# Verify connection
if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")

✅ Connected to Ganache successfully!


In [15]:
# Connect to local Ganache blockchain
ganache_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# Check connection
if not web3.is_connected():
    raise Exception("❌ Not connected to Ganache. Make sure it's running.")

print("✅ Connected to Ganache")

# Replace with your actual contract address from Remix deployment
contract_address = web3.to_checksum_address("0x8f92A6278ee0eE1489767d7bB20BD9Bb35dC5365")

# ABI
abi =[
    {
		"inputs": [],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"anonymous": False,
		"inputs": [
			{
				"indexed": False,
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "enum IoTDataStorage.DataType",
				"name": "dataType",
				"type": "uint8"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"name": "DataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_deviceId",
				"type": "string"
			},
			{
				"internalType": "enum IoTDataStorage.DataType",
				"name": "_dataType",
				"type": "uint8"
			},
			{
				"internalType": "string",
				"name": "_dataValue",
				"type": "string"
			}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "deviceId",
				"type": "string"
			},
			{
				"internalType": "enum IoTDataStorage.DataType",
				"name": "dataType",
				"type": "uint8"
			},
			{
				"internalType": "string",
				"name": "dataValue",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getRecord",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			},
			{
				"internalType": "enum IoTDataStorage.DataType",
				"name": "",
				"type": "uint8"
			},
			{
				"internalType": "string",
				"name": "",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getTotalRecords",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "MAX_ENTRIES",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "owner",
		"outputs": [
			{
				"internalType": "address",
				"name": "",
				"type": "address"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]
# Load the contract
contract = web3.eth.contract(address=contract_address, abi=abi)

# Set the default sender address (typically the first Ganache account)
web3.eth.default_account = web3.eth.accounts[0]

print(f"✅ Connected to Smart Contract at {contract_address}")


✅ Connected to Ganache
✅ Connected to Smart Contract at 0x8f92A6278ee0eE1489767d7bB20BD9Bb35dC5365


In [16]:
total_records = contract.functions.getTotalRecords().call()
print(f"Total IoT records stored: {total_records}")


Total IoT records stored: 105


In [17]:
import pandas as pd


# Retrieve all IoT records
data = []
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    data.append({
        "timestamp": record[0],
        "device_id": record[1],
        "data_type": record[2],
        "data_value": record[3]
    })


In [18]:
# Convert to a DataFrame
df = pd.DataFrame(data)

In [19]:
# Convert timestamp to readable format
df["timestamp"] = pd.to_datetime(df["timestamp"], unit="s")

In [21]:
# Display first few records
print("📦 Raw data preview:")
print(df.head())

📦 Raw data preview:
            timestamp  device_id  data_type data_value
0 2025-06-09 05:47:24  device123          0     26.5°C
1 2025-06-09 05:47:35    TEST001          0     22.5°C
2 2025-06-09 05:49:30    PKG8095          0     8.61°C
3 2025-06-09 05:49:31    PKG7668          0     5.23°C
4 2025-06-09 05:49:32    PKG6401          0     8.59°C


In [22]:
# Extract numeric values from 'data_value' (e.g., "22.5°C" → 22.5)
df["numeric_value"] = df["data_value"].str.extract(r'(\d+\.?\d*)').astype(float)

In [23]:
# Handle missing values (fill with 0)
df.fillna(0, inplace=True)

In [24]:
# Preview cleaned data
print("🧼 Cleaned data preview:")
print(df.head())

🧼 Cleaned data preview:
            timestamp  device_id  data_type data_value  numeric_value
0 2025-06-09 05:47:24  device123          0     26.5°C          26.50
1 2025-06-09 05:47:35    TEST001          0     22.5°C          22.50
2 2025-06-09 05:49:30    PKG8095          0     8.61°C           8.61
3 2025-06-09 05:49:31    PKG7668          0     5.23°C           5.23
4 2025-06-09 05:49:32    PKG6401          0     8.59°C           8.59


In [25]:
# Save the cleaned DataFrame to a CSV file
df.to_csv("cleaned_iot_data.csv", index=False)
print("✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv")

✅ Cleaned IoT data saved successfully as cleaned_iot_data.csv
